
# Getting Started with TSPulse Classification

This notebook demonstrates the usage of a pre-trained TSPulse model for time-series classification task. Refer to [TSPulse](https://arxiv.org/abs/2505.13033) paper for architecture and other details.

Backbone of the pre-trained model is freezed and the classifier head along with the input patch embedding layer is finetuned on the classification dataset.

The pre-trained TSPulse model can be accessed from the [Hugging Face TSPulse Model Repository](https://huggingface.co/ibm-granite/granite-timeseries-tspulse-r1).


## Imports

In [1]:
import math
import os
import tempfile

import numpy as np
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader, random_split
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed
from transformers.data.data_collator import default_data_collator
from transformers.trainer_utils import RemoveColumnsCollator

In [2]:
import warnings


warnings.filterwarnings("ignore")

In [ ]:
import sys
import os

# Get the absolute path of the current notebook's directory.
# This makes the code work no matter where you launch jupyter from.
notebook_dir = os.path.abspath(os.path.dirname(''))

# Assuming the 'tsfm_public' library is in a subdirectory called 'granite-tsfm'
# located in the same root as the notebook.
# This constructs the full path to the library.
project_root = os.path.dirname(notebook_dir) # Go up one level if notebook is in a subfolder
tsfm_public_path = os.path.join(project_root, "..")

# Add the library's parent directory to the system path
# We add the parent so that python can resolve `from tsfm_public...`
if tsfm_public_path not in sys.path:
    print(f"Adding {tsfm_public_path} to system path")
    sys.path.insert(0, tsfm_public_path)

from tsfm_public.models.tspulse import TSPulseForClassification
from tsfm_public.toolkit.dataset import ClassificationDFDataset
from tsfm_public.toolkit.lr_finder import optimal_lr_finder
from tsfm_public.toolkit.time_series_classification_preprocessor import TimeSeriesClassificationPreprocessor
from tsfm_public.toolkit.util import convert_tsfile_to_dataframe

## Data Preprocessing

In [6]:
seed = 42
set_seed(seed)

In [7]:
dataset_name = "BasicMotions"

In [ ]:
# --- Use Relative Paths for Portability ---

# The dataset directory is relative to the current notebook's location.
dataset_dir = "Multivariate_ts"

# Construct the relative paths to the train and test files
path = os.path.join(dataset_dir, dataset_name, f"{dataset_name}_TRAIN.ts")

df_base = convert_tsfile_to_dataframe(
    path,
    return_separate_X_and_y=False,
)

label_column = "class_vals"
input_columns = [f"dim_{i}" for i in range(df_base.shape[1] - 1)]

tsp = TimeSeriesClassificationPreprocessor(
    input_columns=input_columns,
    label_column=label_column,
    scaling=True,
)

tsp.train(df_base)
df_train_prep = tsp.preprocess(df_base)

base_dataset = ClassificationDFDataset(
    df_train_prep,
    id_columns=[],
    timestamp_column=None,
    input_columns=input_columns,
    label_column=label_column,
    context_length=512,
    static_categorical_columns=[],
    stride=1,
    enable_padding=False,
    full_series=True,
)

path = os.path.join(dataset_dir, dataset_name, f"{dataset_name}_TEST.ts")

df_test = convert_tsfile_to_dataframe(
    path,
    return_separate_X_and_y=False,
)

label_column = "class_vals"
input_columns = [f"dim_{i}" for i in range(df_test.shape[1] - 1)]

df_test_prep = tsp.preprocess(df_test)

test_dataset = ClassificationDFDataset(
    df_test_prep,
    id_columns=[],
    timestamp_column=None,
    input_columns=input_columns,
    label_column=label_column,
    context_length=512,
    static_categorical_columns=[],
    stride=1,
    enable_padding=False,
    full_series=True,
)


# creating a validation set

dataset_size = len(base_dataset)
print(dataset_size)
split_valid_ratio = 0.1
val_size = int(split_valid_ratio * dataset_size)  # 10% valid split
train_size = dataset_size - val_size
train_dataset, valid_dataset = random_split(base_dataset, [train_size, val_size])

## Configs for the TSPulse model
### Hyperparameters to Optimize and suggested values :
#### head_reduce_d_model = 1, 2
#### decoder_mode = mix_channel, common_channel
#### head_gated_attention_activation = softmax, sigmoid
#### mask_ratio = 0, 0.3
#### channel_virtual_expand_scale = 1, 2

In [18]:
config_dict = {
    "head_gated_attention_activation": "softmax",
    "channel_virtual_expand_scale": 2,
    "mask_ratio": 0.3,
    "head_reduce_d_model": 1,
    "disable_mask_in_classification_eval": True,
    "fft_time_consistent_masking": True,
    "decoder_mode": "mix_channel",
    "head_aggregation_dim": "patch",
    "head_aggregation": None,
    "loss": "cross_entropy",
    "ignore_mismatched_sizes": True,
}

config_dict["num_input_channels"] = tsp.num_input_channels
config_dict["num_targets"] = df_base["class_vals"].nunique()

## Getting the Pretrained Model with above configs

In [ ]:
model = TSPulseForClassification.from_pretrained(
    "ibm-granite/granite-timeseries-tspulse-r1", revision="tspulse-block-dualhead-512-p16-r1", **config_dict
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
# model = model.to(device).float()
print(device)

In [21]:
# Freezing Backbone except patch embedding layer....

for param in model.backbone.parameters():
    param.requires_grad = False

for param in model.backbone.time_encoding.parameters():
    param.requires_grad = True
for param in model.backbone.fft_encoding.parameters():
    param.requires_grad = True

## Finetuning the classifier head and patch embedding layer

In [22]:
OUT_DIR = "tspulse_finetuned_models/"

In [ ]:
temp_dir = tempfile.mkdtemp()

suggested_lr = None

train_dict = {"per_device_train_batch_size": 32, "num_train_epochs": 200, "eval_accumulation_steps": None}

EPOCHS = train_dict["num_train_epochs"]
BATCH_SIZE = train_dict["per_device_train_batch_size"]
eval_accumulation_steps = train_dict["eval_accumulation_steps"]
NUM_WORKERS = 1
NUM_GPUS = 1

set_seed(42)
if suggested_lr is None:
    lr, model = optimal_lr_finder(
        model,
        train_dataset,
        batch_size=BATCH_SIZE,
    )
    suggested_lr = lr
print("Suggested LR : ", suggested_lr)
finetune_args = TrainingArguments(
    output_dir=temp_dir,
    overwrite_output_dir=True,
    learning_rate=suggested_lr,
    num_train_epochs=EPOCHS,
    do_eval=True,
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_accumulation_steps=eval_accumulation_steps,
    dataloader_num_workers=NUM_WORKERS,
    report_to="tensorboard",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=1,
    logging_dir=os.path.join(OUT_DIR, "output"),  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
)

# Create the early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=100,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.0001,  # Minimum improvement required to consider as improvement
)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=suggested_lr)
scheduler = OneCycleLR(
    optimizer,
    suggested_lr,
    epochs=EPOCHS,
    steps_per_epoch=math.ceil(len(train_dataset) / (BATCH_SIZE * NUM_GPUS)),
)

finetune_trainer = Trainer(
    model=model,
    args=finetune_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
    optimizers=(optimizer, scheduler),
)

# Fine tune
finetune_trainer.train()

## Classification Scores

In [ ]:
predictions_dict = finetune_trainer.predict(test_dataset)
preds_np = predictions_dict.predictions[0]

remove_columns_collator = RemoveColumnsCollator(
    data_collator=default_data_collator,
    signature_columns=["target_values"],
    logger=None,
    description=None,
    model_name="temp",
)

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=remove_columns_collator)
target_list = []
for batch in test_dataloader:
    batch_labels = batch["target_values"].numpy()
    target_list.append(batch_labels)
targets_np = np.concatenate(target_list, axis=0)
test_accuracy = np.mean(targets_np == np.argmax(preds_np, axis=1))
print("test_accuracy : ", test_accuracy)

## Using Classification Pipeline for inference with the finetuned model

In [ ]:
from tsfm_public.toolkit.time_series_classification_pipeline import TimeSeriesClassificationPipeline


pipe = TimeSeriesClassificationPipeline(finetune_trainer.model, feature_extractor=tsp, device=device)

In [ ]:
pipe(df_test)